# **Deep Research with Agent Framework (Azure AI Agents SDK v2)**

This notebook demonstrates an agentic research workflow using **Microsoft Agent Framework** with the **Azure AI Agents SDK v2**. This is an evolution of notebook 04, updated to leverage the latest v2 SDK features including improved agent management, enhanced response handling with `response_format`, and streamlined workflow orchestration.

## Key Features

- **Workflow Orchestration** - Uses Agent Framework's WorkflowBuilder for declarative agent coordination
- **Conditional Routing** - Implements switch-case logic for dynamic workflow paths based on peer review feedback
- **Pure Azure AI Agents** - All agents pre-created in Azure AI Foundry
- **Automated Quality Control** - Peer review agent routes workflow to completion, revision, or data gathering

Built with Microsoft Agent Framework and Azure AI Agents SDK v2

In [1]:
# Environment and dependency setup
import os
import dotenv
dotenv.load_dotenv(".env", override=True)

# Enable nested async event loops (required for Jupyter notebooks)
import nest_asyncio
nest_asyncio.apply()

# Agent Framework imports for workflow orchestration
from agent_framework import WorkflowBuilder, Case, Default

# Project-specific imports (using v2 SDK agents and nodes)
from common.data_models import NextAction
from maf.helper import save_report
from maf.agents_v2 import cleanup_all_agents
from maf.nodes_v2 import (
    planner_executor,
    search_executor,
    summary_executor,
    research_report_executor,
    peer_review_executor,
    to_routing_decision,
    get_next_action,
    handle_complete,
    handle_routing_error,
)

### Creating Azure AI v2 Agents (One-time Setup)

The following cell will **create all Azure AI v2 Agents** required for this workflow. You only need to run this cell **once** to create the agents, then save their IDs to your `.env` file.

After creating the agents, uncomment the fetch agents cell below and comment out this creation cell for subsequent runs.

In [ ]:
# If you're running this workflow for the first time, uncomment the line below to create all agents
# from common.create_azure_ai_agents_v2 import create_agents
# create_agents()

# Update all agent instructions with current date and refined prompts
# from common.create_azure_ai_agents_v2 import update_agents
# update_agents()

## Workflow Setup

The workflow orchestrates multiple agents in a research pipeline using the v2 SDK's executor pattern:

1. **PlannerExecutor** → Creates research plan with subtopics and queries (returns `ResearchPlan` via `response_format`)
2. **SearchExecutor** → Executes Bing searches for each query
3. **SummaryExecutor** → Summarizes search results
4. **ResearchReportExecutor** → Generates comprehensive report (returns `ComprehensiveResearchReport` via `response_format`)
5. **PeerReviewExecutor** → Evaluates report quality (returns `PeerReviewFeedbackMultiChoice` via `response_format`)
6. **RoutingDecision** → Routes to completion, revision, or more data gathering

The peer review agent uses multi-choice output to determine the workflow path:
- `COMPLETE` → Workflow finishes, report is saved
- `REVISE_REPORT` → Loops back to research report executor with feedback
- `GATHER_MORE_DATA` → Loops back to search executor for additional information

In [3]:
# Build the workflow with declarative edge definitions
workflow = (
    WorkflowBuilder()
    .set_start_executor(planner_executor)
    .add_edge(planner_executor, search_executor)
    .add_edge(search_executor, summary_executor)
    .add_edge(summary_executor, research_report_executor)
    .add_edge(research_report_executor, peer_review_executor)
    .add_edge(peer_review_executor, to_routing_decision)
    .add_switch_case_edge_group(
        to_routing_decision,
        [
            Case(condition=get_next_action(NextAction.COMPLETE), target=handle_complete),
            Case(condition=get_next_action(NextAction.REVISE_REPORT), target=research_report_executor),
            Case(condition=get_next_action(NextAction.GATHER_MORE_DATA), target=search_executor),
            Default(target=handle_routing_error),
        ]
    )
    .build()
)

Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Addi

## Workflow Visualization

Generate a visual representation of the agent workflow showing all nodes and routing paths.

In [4]:
# print("Generating workflow visualization...")
# viz = WorkflowViz(workflow)

# try:
#     svg_file = viz.export(format="svg", filename="workflow_graph_agent_framework.svg")
#     print(f"SVG file saved to: {svg_file}")
# except ImportError:
#     print("Tip: Install 'viz' extra to export workflow visualization: pip install agent-framework[viz] --pre")

## Research Query

Define your research topic or question below.

In [5]:
user_query="What are the differences between classical machine learning, deep learning and generative AI?"

## Execute Workflow

Run the complete research workflow using the v2 SDK. The workflow will:
1. Plan the research (structured output via `response_format`)
2. Search for information using Bing grounding
3. Summarize findings
4. Generate a comprehensive report (structured output via `response_format`)
5. Peer review and iterate until quality standards are met
6. Save the final report

The v2 SDK's executor pattern ensures clean separation of concerns, with each executor handling its agent's business logic and automatic response parsing.

In [6]:
try:
    events = await workflow.run(user_query)
    outputs = events.get_outputs()
    final_report = outputs[0]   
    save_report(final_report)

except Exception as e:
    print(f"Error during workflow execution: {e}")
    raise
finally:
    print("\n[Main] Cleaning up agent clients...")
    await cleanup_all_agents()

[AgentLoggingMiddleware] 🚀 PlannerAgent starting...
[AgentLoggingMiddleware] ✅ Agent PlannerAgent completed in 22.21s
[SearchExecutor] Initial search execution from planner
[SearchExecutor] Stored research plan in shared state (6 tasks, 24 queries)
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 search query started...
[SearchExecutor] 🚀 s

In [7]:
from IPython.display import display, Markdown
display(Markdown(final_report))

# The Landscape of Classical Machine Learning, Deep Learning, and Generative AI: Definitions, Techniques, Applications, and Future Directions

## Introduction

Artificial Intelligence (AI) has rapidly transformed from the domain of theoretical research into a technological force permeating every industry, discipline, and aspect of daily life. Central to this transformation are three foundational approaches: classical machine learning (ML), deep learning (DL), and the burgeoning field of generative artificial intelligence (GenAI). While these domains share common mathematical and algorithmic roots, they have diverged in methodologies, applications, and capabilities, each representing distinct milestones in the evolution of data-driven intelligence. This report aims to provide an exhaustive, integrated analysis of these three domains—distilling their definitions, methods, strengths and weaknesses, distinctive and overlapping applications, as well as their interrelationships and future trends. By dissecting these facets, the report not only serves to clarify technical distinctions but also situates these technologies within the broader narrative of AI’s ongoing development and innovation.

## Definitions and Core Concepts

Artificial Intelligence at its broadest refers to the science of creating machines that mimic human-like reasoning, perception, learning, and decision-making. Within this vast field, machine learning is the influential subset wherein algorithms discern patterns from data, providing predictions or decisions without explicit, rule-based programming. Machine learning’s process is fundamentally one of training on historical data, generalizing learnt relationships to unseen scenarios, and refining these models iteratively.

Classical machine learning, often termed “traditional” ML, embodies the earliest and most widely adopted computational methods within this subfield. These models typically leverage statistical algorithms—such as linear regression, decision trees, support vector machines (SVMs), and clustering techniques—that operate on structured, tabular data. The hallmark of classical ML is its reliance on human expertise for feature engineering, wherein the selection and design of input features by domain experts is paramount to model success. Interpretability, simplicity, and computational efficiency are central virtues of this approach, especially in domains requiring transparency and regulatory oversight [1], [2], [4].

Deep learning, by contrast, is a specialized branch within machine learning, harnessing multi-layered neural network architectures to automatically extract and learn hierarchical representations from raw data. These architectures, such as convolutional neural networks (CNNs) for images or transformers for text, can discern complex patterns without manual feature design, thriving on massive datasets and high computational power. The ability of deep learning to handle unstructured data modalities—images, speech, text—has enabled state-of-the-art performance in fields ranging from computer vision to natural language processing [6], [13], [14].

Generative AI represents the most recent and transformative frontier in this continuum. Rather than simply classifying or predicting, generative models such as generative adversarial networks (GANs), variational autoencoders (VAEs), and large language models (LLMs) learn underlying data distributions to generate new, realistic content—be it synthetic images, human-like text, functional code, or even music. Generative AI is not a separate pillar but a domain within deep learning, standing out for its creative, open-ended task orientation and foundational impact on both business and cultural landscapes [9], [10], [12].

These delineations are pivotal, not only clarifying technical boundaries but also illuminating the evolutionary trajectory from human-guided modeling toward increasingly automated, expressive, and creative intelligence. The following sections will unpack the core methodologies and algorithmic families within each domain, their comparative strengths and limitations, real-world applications, and recent waves of innovation.

## Algorithms and Techniques

The diversity of algorithmic paradigms within artificial intelligence is strikingly revealed when contrasting classical machine learning, deep learning, and generative AI. Under the umbrella of classical machine learning, techniques are generally separated into supervised, unsupervised, and reinforcement learning. Supervised methods such as linear regression, logistic regression, SVMs, k-nearest neighbors (k-NN), decision trees, and ensemble models like random forests and gradient boosting, are deployed to predict outcomes from labeled data. These algorithms rely heavily on predefined features, are computationally light, and excel in settings where interpretability and structure predominate [3], [7], [8].

Unsupervised classical approaches—clustering (k-means, hierarchical), dimensionality reduction (principal component analysis), and association rule mining—uncover structure within unlabeled data, facilitating tasks like segmentation, anomaly detection, and visualization. Reinforcement learning at the classical phase employs tabular methods such as Q-learning or SARSA, most effective for constrained environments with tractable state spaces [8], [15]. The strength of classical techniques lies in their robust theoretical foundation, practical tractability, and transparent model dynamics, though their dependence on feature engineering and difficulty with high-dimensional, unstructured data remain limiting [1], [4].

Deep learning diverges sharply by automating the learning of representations from raw, complex data. At its core are deep neural networks (DNNs), which, through layers of interconnected artificial neurons, transform and abstract data to discern intricate relationships. Specialized architectures abound—CNNs process spatial information in images and video; recurrent neural networks (RNNs), including advanced variants such as long short-term memory (LSTM) and gated recurrent units (GRUs), excel at sequential and temporal modeling in language or audio; transformers, employing attention mechanisms, have revolutionized both language and increasingly vision; autoencoders and VAEs facilitate unsupervised representation and generative exploration. Training deep networks leverages sophisticated optimization techniques, regularization schemes (dropout, normalization), and the massive compute afforded by modern hardware [6], [13], [14], [19].

Generative AI, deeply entwined with these advances, employs models that learn data distributions to synthesize new data. VAEs encode inputs into latent spaces, balancing reconstruction fidelity and regularization for smooth, interpretable generation, though outputs can lack sharpness. GANs, in contrast, stage a duel between a generator and a discriminator—leading to the creation of highly realistic images but at the cost of instability and challenging training dynamics. LLMs, built on deep transformers, predict subsequent tokens in text or code streams, with capacities for coherence, context retention, and even rudimentary reasoning. Applications extend to content creation (written and visual media), augmentation of existing datasets for diversity, anomaly detection, pretraining for downstream tasks, and simulation. The immense promise of generative models is intertwined with requirements for unprecedented scales of data and computation, as well as risks of output unreliability and opacity [17], [20], [22].

The interplay and connection between these algorithmic families is increasingly pronounced in modern practice. Hybrid systems frequently deploy deep learning architectures to extract features from unstructured inputs, which are then passed to classical models for structured inference or business logic. Generative models themselves often begin with supervised or self-supervised objectives before transitioning to open-ended, creative tasks. The distinction between approaches is thus often not binary, but rather a reflection of scale, expressiveness, and orientation toward either prediction or generation.

## Applications and Use Cases

The practical reach of classical machine learning, deep learning, and generative AI is both extensive and distinct, with considerable overlap. Classical supervised learning remains the bedrock in business analytics and operations. Classification algorithms are deployed for spam detection in communications, fraud detection in finance, medical diagnoses through structured patient records, customer churn prediction in telecoms, and sentiment analysis of textual reviews. Regression techniques underpin price forecasting in real estate, risk modeling in insurance, outcome prediction in healthcare, and targeted advertising by predicting click-through rates. Unsupervised learning—clustering for customer segmentation, anomaly detection for fraud and manufacturing, dimensionality reduction for visualization—continues to add value where labeled data are sparse or too costly to collect [23], [28], [31].

Cross-industry deployment of classical ML pervades healthcare (diagnosis and risk stratification), finance (credit scoring, segmentation), retail (forecasting, churn analysis), telecoms (anomaly detection, ticket classification), manufacturing (predictive maintenance), marketing (lead scoring, response optimization), and more. The simplicity and interpretability of these models are especially prized in regulated environments where transparency is mandatory [24], [30].

Deep learning’s strength manifests in domains dominated by unstructured or high-dimensional data. In healthcare, deep learning has transformed medical imaging—enabling fast, accurate segmentation and diagnosis of tumors or organs, time-series analysis from wearables, and natural language parsing of electronic records. In financial technology, deep models power advanced fraud detection, credit modeling using heterogeneous data sources, and anomaly detection in cybersecurity. Autonomous vehicles employ deep learning for sensor fusion, perception, and decision systems, while logistics leverage it for route optimization [27], [32], [35].

Retail and e-commerce benefit from deep recommendation engines, personalization systems, and targeted ad placement, while manufacturing and industrial sectors deploy deep models in visual inspection, predictive maintenance, and process analysis. In agriculture, computer vision enables crop and pest monitoring, yield prediction, and resource optimization. Deep learning is also at the core of the “core AI” tasks—computer vision (object detection, facial recognition), speech and audio analysis, and the entire gamut of natural language processing (translation, summarization, chatbots) [25], [33]. Emerging applications extend further into drug discovery, genomic analysis, urban systems, smart city monitoring, and generative content creation for media and creative industries.

Generative AI, a field that has accelerated with the advent of powerful foundation models, is fundamentally transforming creative, analytic, and operational domains. Text generation finds use in drafting emails, reports, code documentation, and even poetry, while image synthesis enables rapid marketing creative, product renderings, and fine art at scale. Audio generation supports music composition, voice synthesis, and podcast editing; video generation powers advertising, promotional content, and cinematic prototypes [29], [36], [38]. In software, generative models autocomplete code, generate functions and tests, refactor, and even translate between languages. Data and analytics professionals use synthetic data for augmentation, anonymization, and scenario simulation, enabling robust model development and privacy preservation.

Industry-specific uses of generative AI are burgeoning—healthcare leverages synthetic medical data and supports drug design, finance utilizes GenAI for report generation and scenario modeling, manufacturing employs generative models for design and documentation, and media and entertainment are increasingly powered by AI-driven writing, visual assets, and video production. Education and training benefit from personalized content creation and simulation, legal and compliance functions use GenAI for policy review and contract analysis. The impact is both quantitative—in improved productivity and measurable return on investment—and qualitative, in enhancing creativity and transforming human–machine collaboration [24], [27].

Integrated solutions increasingly blend all three approaches: a churn prediction system may use classical ML for scoring, deep learning for feature extraction from text or images, and generative AI to craft personalized retention emails. The “right fit” for these technologies is often determined by the data types, required interpretability, scale, and intended impact of an application.

## Strengths and Limitations

The diversity of methodologies across classical ML, deep learning, and generative AI brings a spectrum of strengths and constraints. Classical machine learning is lauded for its effectiveness on small to medium-sized datasets, rapid training and inference on basic hardware, and suitability for regulated domains where interpretability and transparency are paramount. Models such as linear regression, decision trees, and even ensemble techniques provide clear mappings between input features and predictions, enabling experts to trace, explain, and audit their decisions—critical for fields such as healthcare and finance [39], [41], [42]. The operational simplicity, reduced risk of overfitting on limited data, and suitability for edge or real-time applications cement classical ML’s ongoing relevance. Feature engineering, when coupled with domain expertise, can produce highly robust and reliable models.

However, classical ML is bounded by its need for manual feature creation, plateauing performance on highly complex, high-dimensional or unstructured problems, and limited capacity to learn from raw inputs—images, audio, natural language—without substantial preprocessing. As data scales or complexity intensifies, deep learning approaches become necessary for continued improvement.

Deep learning’s most celebrated virtue is the automatic extraction of hierarchical and highly expressive features from raw data. Neural networks can learn subtle and complex patterns, driving state-of-the-art results in computer vision, speech recognition, natural language understanding, and more. Architectures such as transformers have achieved breakthroughs in both generality and transferability—often improving with larger datasets and more powerful hardware. Deep learning models are versatile, capable of being retrained for fresh tasks, and underpinning multimodal fusion across images, text, and audio [44], [48].

Yet, deep learning is not without drawbacks. Models are “data hungry,” often requiring millions of annotated examples for reliable training—an impractical requirement in many domains. Compute intensity is also a factor, necessitating specialized hardware and extensive engineering support. The opaque, “black box” nature of deep models impedes explanation, auditing, and debugging, posing challenges in deployment—especially where accountability and fairness are critical. Overfitting, brittleness to distribution shifts or adversarial attacks, and complex engineering requirements add layers of difficulty. Deep models may also struggle with systematic reasoning or logical task structures, where hybrid approaches or rule-based methods may excel [47], [50].

Generative AI’s strength lies in scalable, creative output. These models can generate rich text, images, audio, and video from brief prompts, vastly expanding the scope of what AI can automate. Generative models facilitate data augmentation, support few- or zero-shot learning, and enable simulation or scenario planning. Productivity gains are evident across writing, coding, design, and content production workflows. However, these gains come with novel risks: Generative models can “hallucinate” false but plausible outputs, inherit or amplify bias and discrimination from training data, and produce convincing misinformation or deepfakes. Intellectual property and privacy concerns stem from the ambiguous provenance of training data, and the models’ ability to memorize or regurgitate proprietary information [56], [59]. High computational infrastructure requirements restrict pretraining and large-scale deployment to major institutions. Control, alignment, and ethical governance are active frontiers, as models can be “jailbroken”, engage in unsafe behaviors, or present risks of misuse.

Across all three domains, selection of methodologies must be attuned to the core requirements of the application—data scale and type, interpretability, performance, resource constraints, and ethical expectations.

## Relationships and Interconnections

A nuanced understanding of AI landscapes reveals not only differences, but a deep continuity and hierarchy among classical ML, deep learning, and generative AI. Conceptually, classical ML forms the statistical and algorithmic foundation for the field. Deep learning is a specialized and highly expressive branch within machine learning, automating the representation learning that classical ML accomplishes manually. Generative AI, in turn, is a modern subset within deep learning, with models focused on synthesizing novel data [61], [68].

Classical ML is typified by explicit, human-designed features and interpretable, shallow models. Deep learning extends this by stacking multiple neural layers, automating feature discovery, and enabling models to learn directly from raw inputs. Many classical workflows—train–test splits, hyperparameter tuning, loss function design and regularization—are preserved in deep learning, but the focus shifts from feature engineering to architectural design and scalable training protocols. While classical models excel in interpretability and efficiency, deep learning sacrifices transparency for power, versatility, and ability to handle unstructured, complex modalities [62], [66].

Generative AI builds on deep learning advances, leveraging unsupervised or self-supervised architectures to transcend prediction tasks, moving into generation and simulation. Although not all deep learning models are generative, nearly all generative AI models are deep neural networks—whether GANs, VAEs, or transformers. The shared principles of optimization, generalization, and regularization echo across the hierarchy, while ever-larger and more expressive models expand task boundaries [63], [70].

Hybrid pipelines are increasingly common, with deep learning architectures providing automatic feature extraction for classical ML models, or generative models serving as pretraining scaffolds. Organizations may begin with interpretable classical models for transparency, scaling to deep learning as data and complexity grow, and then integrating generative AI as creative or content tasks emerge. Thus, the domains transcend simple segmentation, forming a “continuum” of capability linked by foundational theory, descendant architectures, and shared challenges [65], [69].

## Recent Developments and Future Trends

The past several years have witnessed an acceleration of technical and societal innovation across all three branches of AI, particularly generative AI. Large, multimodal foundation models now integrate text, vision, audio, and video, with expanded context windows and improved reasoning. Hardware advances such as GPUs and purpose-built AI accelerators drive efficiency, enabling increasingly sophisticated and accessible models [73], [81]. Open-weight models, agentic systems capable of autonomous decision-making and tool use, and deepening specialization of generative approaches for healthcare, coding, and the creative arts are redefining industry norms and workflows.

Governance and ethical accountability have become pressing concerns. New frameworks, such as the European Union AI Act, are shaping regulatory landscapes, driving advances in transparency, safety, and risk management. Intellectual property and privacy issues have ignited debates about data provenance and usage, while mitigation strategies for misinformation—watermarking, provenance tracking, detection—grow in urgency as generative models proliferate. The future will be marked by labor market disruption, policy evolution, and the imperative to balance innovation with oversight [75], [77].

Generative AI is set to expand into multimodal and embodied domains, with integration of sensor data for AR/VR, robotics, and real-world interaction. Agentic AI—persistent, autonomous systems with memory, learning and multi-agent cooperation—will increasingly reshape enterprise operations. Hyper-personalization, powered by advanced user modeling and edge computing, will enable granular customization across healthcare, retail, and entertainment. Efforts toward “green AI”—efficiency, sustainability, and edge deployment—will address the environmental footprint of large models [79], [82]. Policy and technical advances in safety and alignment will provide scalable methods to ensure that AI models reflect human and societal values, with domain-specific evaluation, auditing, and ongoing governance. Across sectors, generative and hybrid AI will be embedded with strong oversight, emphasizing human–AI collaboration, domain-specific value, and continuous learning.

Classical machine learning, meanwhile, continues to modernize. AutoML pipelines, interpretable and constraint-aware modeling, and robust MLOps integration are democratizing model deployment and maintenance. Edge and federated learning preserve classical algorithms’ relevance in privacy-sensitive, resource-constrained settings. Hybrid systems will leverage both classical and deep techniques for efficiency and performance, with ongoing research in quantum and optimization seeking new horizons [74], [76].

Deep learning advances persist with foundation models as unified solutions for varied tasks, improved training efficiency, and architectures such as graph neural networks, capsule networks, and neuro-symbolic hybrids extending flexibility and power. Diffusion models and GANs are expanding creative and simulation capacities, while reinforcement learning gains a foothold in agentic decision-making systems. Systems-level optimization for speed, energy, and scalability remains critical, broadening deployment possibilities in healthcare, engineering, and science. Governance, explainability, and fairness are imperative for robust, ethical implementation.

The future is thus one of diversification and collaboration across model types, modalities, and functions—characterized by multimodal foundation models, agentic workflows, efficient and sustainable design, ethical alignment, and comprehensive governance. Successful innovation will hinge on domain-specific value creation, hybrid and human–AI partnerships, and the continuous refinement of both technical and ethical paradigms.

## Conclusion

The evolution from classical machine learning to deep learning and generative AI constitutes not merely a progression of technical sophistication but a transformation in the creative and operational boundaries of artificial intelligence. Classical ML, with its structured approach, interpretability, and efficiency, remains foundational for countless practical applications. Deep learning, powered by automatic representation learning, large-scale data, and adaptable architectures, leads the way in perception and sequence modeling. Generative AI, built on the advances of deep learning, is unleashing new waves of productivity, creativity, and risk—heralding both promising and challenging futures across sectors.

As AI’s influence deepens, the distinctions and interconnections among these approaches will remain essential for informed development, strategic deployment, and responsible governance. The future promises not only continued technical innovation but a reimagining of the interface between human capabilities and artificial intelligence—a story best understood through both their differences and the principles that unite them.

## References

[1] Introduction to Machine Learning - GeeksforGeeks, https://www.geeksforgeeks.org/machine-learning/introduction-machine-learning/

[2] Introduction to Machine Learning Concepts - Training, https://learn.microsoft.com/en-us/training/modules/fundamentals-machine-learning/

[3] Deep learning vs. classic machine learning: differences you should know ..., https://iartificial.blog/en/learning/Deep-learning-vs.-classic-machine-learning%3A-differences-you-should-know/

[4] Classical Machine Learning Principles and Methods, https://link.springer.com/chapter/10.1007/978-1-4842-8692-0_1

[6] Deep Learning vs. Traditional Machine Learning: Which is Better?, https://www.alliancetek.com/blog/post/2025/03/11/deep-learning-vs-traditional-ml.aspx

[7] Classic and Adaptive machines - GeeksforGeeks, https://www.geeksforgeeks.org/machine-learning/classic-and-adaptive-machines/

[8] Generative artificial intelligence - Wikipedia, https://en.wikipedia.org/wiki/Generative_artificial_intelligence

[9] Generative AI Tutorial - GeeksforGeeks, https://www.geeksforgeeks.org/artificial-intelligence/generative-ai-tutorial/

[10] What is Generative AI (GenAI)? Definition, How It Works, Applications ..., https://www.sciencenewstoday.org/what-is-generative-ai-genai-definition-how-it-works-applications-and-future-impact

[12] What Is Generative AI? Definition, Applications, and Impact, https://www.coursera.org/gb/articles/what-is-generative-ai

[13] Deep Learning vs Machine Learning - Difference Between Data ..., https://aws.amazon.com/compare/the-difference-between-machine-learning-and-deep-learning/

[14] Difference Between Machine Learning and Deep Learning, https://www.geeksforgeeks.org/artificial-intelligence/difference-between-machine-learning-and-deep-learning/

[15] Deep Learning vs Machine Learning: Key Differences Explained, https://csce.engr.tamu.edu/blog/deep-learning-vs-machine-learning/

[17] Classic machine learning methods - arXiv.org, https://arxiv.org/pdf/2310.11470

[19] Generative AI Models 2025: GANs, VAEs & LLMs - futurense.com, https://futurense.com/blog/generative-ai-models

[20] Random forest - Wikipedia, https://en.wikipedia.org/wiki/Random_forest

[22] Generative AI Techniques and Models | SpringerLink, https://link.springer.com/chapter/10.1007/978-3-031-82062-5_3

[23] Deep learning architectures - IBM Developer, https://developer.ibm.com/articles/cc-machine-learning-deep-learning-architectures/

[24] Understanding the Core Generative AI algorithms - Ksolves, https://www.ksolves.com/blog/artificial-intelligence/mastering-generative-ai-vaes-gans-llms-explored

[25] Deep learning - Wikipedia, https://en.wikipedia.org/wiki/Deep_learning

[27] Supervised vs Unsupervised vs Reinforcement Learning, https://www.geeksforgeeks.org/machine-learning/supervised-vs-reinforcement-vs-unsupervised/

[28] Types of Machine Learning: Supervised, Unsupervised & Reinforcement, https://www.sanfoundry.com/types-of-machine-learning-supervised-unsupervised-reinforcement/

[29] Review of Deep Learning Algorithms and Architectures, https://ieeexplore.ieee.org/document/8694781

[30] Comprehensive Review of Generative artificial ... - ScienceDirect, https://www.sciencedirect.com/science/article/pii/S1877050925017326

[31] A Comprehensive Review of Deep Learning: Architectures, Recent Advances ..., https://www.mdpi.com/2078-2489/15/12/755

[32] Generative Models in AI: A Comprehensive Comparison of GANs and VAEs ..., https://www.geeksforgeeks.org/deep-learning/generative-models-in-ai-a-comprehensive-comparison-of-gans-and-vaes/

[33] Neural network (machine learning) - Wikipedia, https://en.wikipedia.org/wiki/Neural_network_(machine_learning)

[35] Artificial intelligence - Wikipedia, https://en.wikipedia.org/wiki/Artificial_intelligence

[36] Deep Learning: Concepts and Architectures | SpringerLink, https://link.springer.com/book/10.1007/978-3-030-31756-0

[38] Deep learning: Historical overview from inception to actualization ..., https://www.sciencedirect.com/science/article/pii/S1568494625006891

[39] The Power of Generative AI: A Review of Requirements, Models, Input ..., https://www.mdpi.com/1999-5903/15/8/260

[41] AI vs ML vs DL vs GenAI: Key Differences Explained - FS.com, https://www.fs.com/blog/artificial-intelligence-vs-machine-learning-vs-deep-learning-vs-generative-ai-key-differences-explained-24134.html

[42] Deep learning: systematic review, models, challenges, and research ..., https://link.springer.com/article/10.1007/s00521-023-08957-4

[44] Comparing Deep Learning and Traditional Machine Learning, https://theceoviews.com/comparing-deep-learning-and-traditional-machine-learning/

[47] Machine learning and generative AI: What are they good for in 2025?, https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-and-generative-ai-what-are-they-good-for

[48] Ethical Challenges and Solutions of Generative AI: An ... - MDPI, https://www.mdpi.com/2227-9709/11/3/58

[50] Deep learning vs. classic machine learning: differences you should know ..., https://iartificial.blog/en/learning/Deep-learning-vs.-classic-machine-learning%3A-differences-you-should-know/

[56] Gen AI vs Machine Learning vs Deep Learning - DataSpace Academy, https://dataspaceacademy.com/blog/genai-vs-machine-learning-vs-deep-learning-a-comparative-study

[59] Governance of Generative AI | Policy and Society | Oxford Academic, https://academic.oup.com/policyandsociety/article/44/1/1/7997395

[61] Exploring the Differences: Deep Learning vs. Traditional Machine ..., https://distributedrobotics.com/exploring-the-differences-deep-learning-vs-traditional-machine-learning/

[62] Difference between AI, ML, LLM, and generative AI - Toloka, https://toloka.ai/blog/difference-between-ai-ml-llm-and-generative-ai/

[63] Deep Learning vs Generative AI: Understanding the Key Differences - eWeek, https://www.eweek.com/artificial-intelligence/generative-ai-vs-deep-learning/

[65] Machine learning - Wikipedia, https://en.wikipedia.org/wiki/Machine_learning

[66] Generative Deep Learning - David Foster - Google Books, https://books.google.com/books/about/Generative_Deep_Learning.html?id=Bkq8EAAAQBAJ

[68] Deep Learning vs Machine Learning: Key Differences, https://labelyourdata.com/articles/deep-learning-vs-machine-learning

[69] Generative AI vs Machine Learning - GeeksforGeeks, https://www.geeksforgeeks.org/artificial-intelligence/generative-ai-vs-machine-learning/

[70] Generative AI and Large Language Models - ScienceDirect, https://www.sciencedirect.com/science/article/pii/S1877050924027492

[73] Machine learning trend analysis in 2025 | Cambridge Advance Online, https://advanceonline.cam.ac.uk/blog/machine-learning-trend-analysis

[74] Recent Advancements in Deep Learning: An In-Depth Analysis of Emerging ..., https://www.researchgate.net/profile/George-Christopher-5/publication/386827127_Recent_Advancements_in_Deep_Learning_An_In-Depth_Analysis_of_Emerging_Trends_and_Techniques_in_Machine_Learning/links/675979c405bf5b3e924dc1fa/Recent-Advancements-in-Deep-Learning-An-In-Depth-Analysis-of-Emerging-Trends-and-Techniques-in-Machine-Learning.pdf

[75] State of Generative AI in the Enterprise 2024 | Deloitte US, https://www.deloitte.com/us/en/what-we-do/capabilities/applied-artificial-intelligence/content/state-of-generative-ai-in-enterprise.html

[76] Recent Advances and Future Perspectives | SpringerLink, https://link.springer.com/chapter/10.1007/978-981-97-8019-8_11

[77] The Defining Moments in Generative AI From 2024 - dataiku.com, https://www.dataiku.com/stories/blog/the-defining-moments-in-generative-ai-from-2024

[79] Emerging trends and future directions in machine learning and deep ..., http://deepscienceresearch.com/index.php/dsr/catalog/download/6/51/348?inline=1

[81] Developments in Artificial Intelligence and Machine Learning: Recent ..., https://www.espjournals.org/IJAST/2024/Volume2-Issue3/IJAST-V2I3P101.pdf

[82] Recent Advances in Generative AI and Large Language Models: Current ..., https://arxiv.org/abs/2407.14962
